<a href="https://colab.research.google.com/github/mahaairshad/CE888_si19783/blob/master/FBCSP/FBCSP_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fname_S1='ParsedMEGData_P5.mat'
fname_S2='ParsedMEGData_P5_S2.mat'

In [0]:
download = drive.CreateFile({'id': '1R72DvIB6ni4pkqxxwOpSGnvF480-gFZZ'})
download.GetContentFile(fname_S1)
download = drive.CreateFile({'id': '1vlhrL0rUnrgutOgJeZlkfIwOcONDNf_z'})
download.GetContentFile(fname_S2)

In [0]:
#Import Libraries
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.signal import butter, sosfilt, sosfreqz
from numpy import linalg as LA
from scipy import linalg as LAS
from sklearn.model_selection import GridSearchCV    # Import library for grid search
from sklearn import svm                             # Import SVM model library
from sklearn import metrics                         # Evaluating Accuracy
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import cohen_kappa_score       # Evaluating Kappa score
from sklearn.model_selection import cross_val_score # Cross Validation
from sklearn.metrics import make_scorer

### 1. Access dataset

In [0]:
# Loading single P's dataset
# s : (1,1)
# x : (3500,300,200)
# y : (1,200)
# c : ~(300,1)
# ci : ~(300,1)
# classInfo : (4,2)

In [0]:
annotsS1 = loadmat(fname_S1)
annotsS2 = loadmat(fname_S2)

In [10]:
mdata1 = annotsS1['MEGdata']
mtype1 = mdata1.dtype
ndata1 = {n: mdata1[n][0,0] for n in mtype1.names}
ndata1

{'c': array([[array(['MEG0111'], dtype='<U7')],
        [array(['MEG0112'], dtype='<U7')],
        [array(['MEG0113'], dtype='<U7')],
        [array(['MEG0121'], dtype='<U7')],
        [array(['MEG0122'], dtype='<U7')],
        [array(['MEG0123'], dtype='<U7')],
        [array(['MEG0131'], dtype='<U7')],
        [array(['MEG0132'], dtype='<U7')],
        [array(['MEG0133'], dtype='<U7')],
        [array(['MEG0141'], dtype='<U7')],
        [array(['MEG0142'], dtype='<U7')],
        [array(['MEG0143'], dtype='<U7')],
        [array(['MEG0211'], dtype='<U7')],
        [array(['MEG0212'], dtype='<U7')],
        [array(['MEG0213'], dtype='<U7')],
        [array(['MEG0221'], dtype='<U7')],
        [array(['MEG0222'], dtype='<U7')],
        [array(['MEG0223'], dtype='<U7')],
        [array(['MEG0231'], dtype='<U7')],
        [array(['MEG0232'], dtype='<U7')],
        [array(['MEG0233'], dtype='<U7')],
        [array(['MEG0241'], dtype='<U7')],
        [array(['MEG0242'], dtype='<U7')],
      

In [11]:
s_sess1=ndata1['s']
x_sess1=ndata1['x']
y_sess1=ndata1['y']
c_sess1=ndata1['c']
ci_sess1=ndata1['ci']
class_info_Sess1=ndata1['classInfo']
#class_info_Sess1=
print(s_sess1.shape,'\n',x_sess1.shape,'\n', y_sess1.shape, '\n', c_sess1.shape, '\n',ci_sess1.shape)

(1, 1) 
 (3500, 297, 200) 
 (1, 200) 
 (297, 1) 
 (297, 1)


In [12]:
mdata2 = annotsS2['MEGdata']
mtype2 = mdata2.dtype
ndata2 = {n: mdata2[n][0,0] for n in mtype2.names}
ndata2

{'c': array([[array(['MEG0111'], dtype='<U7')],
        [array(['MEG0112'], dtype='<U7')],
        [array(['MEG0113'], dtype='<U7')],
        [array(['MEG0121'], dtype='<U7')],
        [array(['MEG0122'], dtype='<U7')],
        [array(['MEG0123'], dtype='<U7')],
        [array(['MEG0131'], dtype='<U7')],
        [array(['MEG0132'], dtype='<U7')],
        [array(['MEG0133'], dtype='<U7')],
        [array(['MEG0141'], dtype='<U7')],
        [array(['MEG0142'], dtype='<U7')],
        [array(['MEG0143'], dtype='<U7')],
        [array(['MEG0211'], dtype='<U7')],
        [array(['MEG0212'], dtype='<U7')],
        [array(['MEG0213'], dtype='<U7')],
        [array(['MEG0221'], dtype='<U7')],
        [array(['MEG0222'], dtype='<U7')],
        [array(['MEG0223'], dtype='<U7')],
        [array(['MEG0231'], dtype='<U7')],
        [array(['MEG0232'], dtype='<U7')],
        [array(['MEG0233'], dtype='<U7')],
        [array(['MEG0241'], dtype='<U7')],
        [array(['MEG0242'], dtype='<U7')],
      

In [13]:
s_sess2=ndata2['s']
x_sess2=ndata2['x']
y_sess2=ndata2['y']
c_sess2=ndata2['c']
ci_sess2=ndata2['ci']
class_info_Sess2=ndata2['classInfo']
#class_info_Sess2=
print(s_sess2.shape,'\n',x_sess2.shape,'\n', y_sess2.shape, '\n', c_sess2.shape, '\n',ci_sess2.shape)

(1, 1) 
 (3500, 297, 200) 
 (1, 200) 
 (297, 1) 
 (297, 1)


### 2. Considering the common channels from Session1 and Session2

In [14]:
import re
chnl_names=c_sess1.reshape(len(c_sess1),)
chnl_names=str(chnl_names)
chnl_names=re.findall('\[\'.*?\'\]',chnl_names)
len(chnl_names)
channel=[]
for i in range (len(chnl_names)):
    k=str(chnl_names[i])[2:-2]
    channel.append(k)
for i in range (len(channel)):
    channel[i]=re.sub(r'(?<=[.G])(?=[^\s])', r' ', channel[i])
len(channel)

297

In [15]:
exclude_chs=['MEG 1633',
 'MEG 1632',
 'MEG 1631',
 'MEG 1643',
 'MEG 1642',
 'MEG 1641',
 'MEG 1713',
 'MEG 1712',
 'MEG 1711',
 'MEG 1723',
 'MEG 1722',
 'MEG 1721',
 'MEG 1733',
 'MEG 1732',
 'MEG 1731',
 'MEG 1743',
 'MEG 1742',
 'MEG 1741',
 'MEG 1833',
 'MEG 1832',
 'MEG 1831',
 'MEG 1843',
 'MEG 1842',
 'MEG 1841',
 'MEG 1913',
 'MEG 1912',
 'MEG 1911',
 'MEG 1923',
 'MEG 1922',
 'MEG 1921',
 'MEG 1933',
 'MEG 1932',
 'MEG 1931',
 'MEG 1943',
 'MEG 1942',
 'MEG 1941',
 'MEG 2013',
 'MEG 2012',
 'MEG 2011',
 'MEG 2023',
 'MEG 2022',
 'MEG 2021',
 'MEG 2033',
 'MEG 2032',
 'MEG 2031',
 'MEG 2043',
 'MEG 2042',
 'MEG 2041',
 'MEG 2113',
 'MEG 2112',
 'MEG 2111',
 'MEG 2123',
 'MEG 2122',
 'MEG 2121',
 'MEG 2133',
 'MEG 2132',
 'MEG 2131',
 'MEG 2143',
 'MEG 2142',
 'MEG 2141',
 'MEG 2233',
 'MEG 2232',
 'MEG 2231',
 'MEG 2243',
 'MEG 2242',
 'MEG 2241',
 'MEG 2313',
 'MEG 2312',
 'MEG 2311',
 'MEG 2323',
 'MEG 2322',
 'MEG 2321',
 'MEG 2333',
 'MEG 2332',
 'MEG 2331',
 'MEG 2343',
 'MEG 2342',
 'MEG 2341',
 'MEG 2433',
 'MEG 2432',
 'MEG 2431',
 'MEG 2443',
 'MEG 2442',
 'MEG 2441',
 'MEG 2513',
 'MEG 2512',
 'MEG 2511',
 'MEG 2523',
 'MEG 2522',
 'MEG 2521',
 'MEG 2533',
 'MEG 2532',
 'MEG 2531',
 'MEG 2543',
 'MEG 2542',
 'MEG 2541',
 'MEG 2633',
 'MEG 2632',
 'MEG 2631',
 'MEG 1533',
 'MEG 1532',
 'MEG 1531',
 ]
exclude_chs=np.asarray(exclude_chs)
exclude_chs.shape

(102,)

In [0]:
cmn_chnls=np.intersect1d(ci_sess1,ci_sess2)
cmn_chnls=cmn_chnls.reshape(len(cmn_chnls),1) #reshaping so cmn_chnls, sess1_x and sess2_x have same shape

In [17]:
channel=np.asarray(channel)
channel=channel.reshape(len(channel),1)
ids_of_chs=np.concatenate((cmn_chnls,channel), axis=1)
ids_of_chs.shape

(297, 2)

In [0]:
valid_chnl_ind1=np.array([],dtype=int)
valid_chnl_ind2=np.array([],dtype=int)

for i in range(0,len(ci_sess1)):    #Stores the index of common channels b/w sess1_x and cmn_chnls
    for j in range(0,len(ids_of_chs)):
        if (ci_sess1[i,0]==int(ids_of_chs[j,0])) and (ids_of_chs[j,1] not in exclude_chs):
            valid_chnl_ind1=np.append(valid_chnl_ind1,i)
            
for i in range(0,len(ci_sess2)):    #Stores the index of common channels b/w sess2_x and cmn_chnls
    for j in range(0,len(ids_of_chs)):
        if (ci_sess2[i,0]==int(ids_of_chs[j,0])) and (ids_of_chs[j,1] not in exclude_chs):
            valid_chnl_ind2=np.append(valid_chnl_ind2,i)

In [19]:
valid_chnl_ind1.shape

(199,)

In [0]:
data1=x_sess1[:,valid_chnl_ind1,:] #(samples,channels,trials)
data2=x_sess2[:,valid_chnl_ind2,:] #(samples,channels,trials)

In [0]:
#Comibining session1 and session2 data (along 3rd axis(trials))
MEGdata_s= s_sess1
MEGdata_x= np.concatenate((data1,data2), axis=2)
MEGdata_y= np.concatenate((y_sess1,y_sess2), axis=1)

In [0]:
MEGdata_c= c_sess1[valid_chnl_ind1]
MEGdata_ci= cmn_chnls.copy()

In [23]:
print(MEGdata_s.shape,'\n',MEGdata_x.shape,'\n', MEGdata_y.shape, '\n', MEGdata_c.shape, '\n',MEGdata_ci.shape)

(1, 1) 
 (3500, 199, 400) 
 (1, 400) 
 (199, 1) 
 (297, 1)


### 3. Implementation of Butterworth Filter

In [0]:
Class_prob=np.array([[1, 2], #Hand Foot
                  [1, 3], #Hand Word
                  [1, 4], #Hand Sub
                  [2, 3], #Foot Word
                  [2, 4], #Foot Sub
                  [3, 4] #Foot Sub
])

In [29]:
Classes[1]

2

In [0]:
 for class_type in range (len(Class_prob)):
  #Selecting Frequency band, Sampling range, Base range(noise) and Classes
  FreqBand1=np.array([8, 12]) #Freqband
  FreqBand2=np.array([13, 17])
  SampRangeSec=np.array([2.5, 6.])#actTs
  BaseRangeSec=np.array([1, 2]) #baseTs
  Classes=Class_prob[class_type] #class1 class2

  SampRange=s_sess1*SampRangeSec
  SampRangeind=np.arange(1250,3000,1,dtype=int)#saves index for given range of time
  BaseRange=s_sess1*BaseRangeSec
  BaseRangeind=np.arange(500,1000,1,dtype=int)#saves index for given range of time

  #Separating data in classification of interest
  cMEGdata_y_ind=np.array([],dtype=int)

  for i in range(0,MEGdata_y.size): 
      if (MEGdata_y[0,i]==Classes[0])or(MEGdata_y[0,i]==Classes[1]):
          cMEGdata_y_ind=np.append(cMEGdata_y_ind,i)

  cMEGdata_x=MEGdata_x[:,:,cMEGdata_y_ind] #(samples,channels,trials)
  cMEGdata_y=MEGdata_y[0,cMEGdata_y_ind]

  def butter_bandpass(lowcut, highcut, fs, order):
          nyq = 0.5 * fs
          low = lowcut / nyq
          high = highcut / nyq
          sos = butter(order, [low, high], analog=True, btype='bandpass', output='sos')
          return sos

  def butter_bandpass_filter(data, lowcut, highcut, fs, order):
          sos = butter_bandpass(lowcut, highcut, fs, order=order)
          y = sosfilt(sos, data)
          return y

  #(samples,channels,trials)--> (channels,trials,samples)
  bMEGdata_x_1=np.zeros((cMEGdata_x[0,:,0].size,cMEGdata_x[0,0,:].size,SampRangeind.size))
  bMEGdata_x_2=np.zeros((cMEGdata_x[0,:,0].size,cMEGdata_x[0,0,:].size,SampRangeind.size))

  for trial in range(0,bMEGdata_x_1[0,:,0].size): #(channels,trials,samples) size of trial
      for channel in range(0,bMEGdata_x_1[:,0,0].size): #(channels,trials,samples) size of channel
          temp=cMEGdata_x[:,channel,trial] #(samples,channels,trials)
          temp=temp-np.mean(temp[BaseRangeind])
          f_temp1=butter_bandpass_filter(temp,FreqBand1[0],FreqBand1[1],MEGdata_s[0],4)
          f_temp1=f_temp1[SampRangeind]
          f_temp2=butter_bandpass_filter(temp,FreqBand2[0],FreqBand2[1],MEGdata_s[0],4)
          f_temp2=f_temp2[SampRangeind]
          bMEGdata_x_1[channel,trial,:]=f_temp1 #(channels,trials,samples)
          bMEGdata_x_2[channel,trial,:]=f_temp2

  ### 4.Separating train and validation data

  # Splitting train/test data in 50:50
  size_Train_Test=int(bMEGdata_x_1[0,:,0].size/2)
  bMEGdata_x_Train_1=bMEGdata_x_1[:,0:size_Train_Test,:]
  bMEGdata_y_Train=cMEGdata_y[0:size_Train_Test]
  bMEGdata_x_Test_1=bMEGdata_x_1[:,size_Train_Test:(size_Train_Test*2),:]
  bMEGdata_y_Test=cMEGdata_y[size_Train_Test:(size_Train_Test*2)]

  c1_bMEGdata_y_Train_ind=np.array([],dtype=int) #Getting index for class1 and class2 data
  c2_bMEGdata_y_Train_ind=np.array([],dtype=int)

  for i in range(0,bMEGdata_y_Train.size): 
      if (bMEGdata_y_Train[i]==Classes[0]):
          c1_bMEGdata_y_Train_ind=np.append(c1_bMEGdata_y_Train_ind,i)
      else:
          c2_bMEGdata_y_Train_ind=np.append(c2_bMEGdata_y_Train_ind,i)

  # x and y values separated for class and 2
  c1_bMEGdata_x_Train_1=bMEGdata_x_Train_1[:,c1_bMEGdata_y_Train_ind,:]
  c1_bMEGdata_y_Train=bMEGdata_y_Train[c1_bMEGdata_y_Train_ind]
  c2_bMEGdata_x_Train_1=bMEGdata_x_Train_1[:,c2_bMEGdata_y_Train_ind,:]
  c2_bMEGdata_y_Train=bMEGdata_y_Train[c2_bMEGdata_y_Train_ind]

  # Merging trial and sample dimensions into one: (channels,trials,samples)-->(channels,trials*samples)
  c1_feat_Train_1=c1_bMEGdata_x_Train_1.reshape((c1_bMEGdata_x_Train_1[:,0,0].size,(c1_bMEGdata_x_Train_1[0,:,0].size*c1_bMEGdata_x_Train_1[0,0,:].size)))
  c2_feat_Train_1=c2_bMEGdata_x_Train_1.reshape((c2_bMEGdata_x_Train_1[:,0,0].size,(c2_bMEGdata_x_Train_1[0,:,0].size*c2_bMEGdata_x_Train_1[0,0,:].size)))


  # Splitting train/test data in 50:50
  size_Train_Test=int(bMEGdata_x_2[0,:,0].size/2)
  bMEGdata_x_Train_2=bMEGdata_x_2[:,0:size_Train_Test,:]
  bMEGdata_y_Train=cMEGdata_y[0:size_Train_Test]
  bMEGdata_x_Test_2=bMEGdata_x_2[:,size_Train_Test:(size_Train_Test*2),:]
  bMEGdata_y_Test=cMEGdata_y[size_Train_Test:(size_Train_Test*2)]

  c1_bMEGdata_y_Train_ind=np.array([],dtype=int) #Getting index for class1 and class2 data
  c2_bMEGdata_y_Train_ind=np.array([],dtype=int)

  for i in range(0,bMEGdata_y_Train.size): 
      if (bMEGdata_y_Train[i]==Classes[0]):
          c1_bMEGdata_y_Train_ind=np.append(c1_bMEGdata_y_Train_ind,i)
      else:
          c2_bMEGdata_y_Train_ind=np.append(c2_bMEGdata_y_Train_ind,i)

  # x and y values separated for class and 2
  c1_bMEGdata_x_Train_2=bMEGdata_x_Train_2[:,c1_bMEGdata_y_Train_ind,:]
  c1_bMEGdata_y_Train=bMEGdata_y_Train[c1_bMEGdata_y_Train_ind]
  c2_bMEGdata_x_Train_2=bMEGdata_x_Train_2[:,c2_bMEGdata_y_Train_ind,:]
  c2_bMEGdata_y_Train=bMEGdata_y_Train[c2_bMEGdata_y_Train_ind]

  # Merging trial and sample dimensions into one: (channels,trials,samples)-->(channels,trials*samples)
  c1_feat_Train_2=c1_bMEGdata_x_Train_2.reshape((c1_bMEGdata_x_Train_2[:,0,0].size,(c1_bMEGdata_x_Train_2[0,:,0].size*c1_bMEGdata_x_Train_2[0,0,:].size)))
  c2_feat_Train_2=c2_bMEGdata_x_Train_2.reshape((c2_bMEGdata_x_Train_2[:,0,0].size,(c2_bMEGdata_x_Train_2[0,:,0].size*c2_bMEGdata_x_Train_2[0,0,:].size)))

  ### 5. Implementation of CSP

  def CSP_projectionMat(class1, class2):
          #1 Find spatial covariance for both classes of data
          C1= np.matmul(class1,np.transpose(class1))/np.trace(np.matmul(class1,np.transpose(class1)))
          C2= np.matmul(class2,np.transpose(class2))/np.trace(np.matmul(class2,np.transpose(class2)))
          #2 Find composite spatial covariance
          Cc=C1+C2
          #3 Find eigen vectors and eigen values for Cc
          eigvalCc, eigvecCc= LA.eig(Cc)
          ## form eigval & eigvec in descending order and form diagonal matrix for eigval
          temp=np.sort(eigvalCc)  #sorts in ascending
          eigvalCc=temp[::-1]     #sorts in descending
          Cc_ind=np.argsort(temp) #finds arg for asc
          Cc_ind=Cc_ind[::-1]     #finds arg for des
          eigvalCc=np.diag(eigvalCc) #form diagonal matrix
          eigvecCc=eigvecCc[:,Cc_ind] #finds respective eigen vectors for the eig values
          #4 Find whitening transform
          P=np.matmul(LA.inv(eigvalCc),np.transpose(eigvecCc))
          P=np.sqrt(np.square(P)) #getting rid of the -ve values for simplicity here(MUST BE REMOVED)
          P=np.sqrt(P)
          #5 Find S1 and S2 for C1 and C2
          S1=np.matmul(P,C1) #S1=PC1P'
          S1=np.matmul(S1,np.transpose(P))
          S2=np.matmul(P,C2) #S2=PC2P'
          S2=np.matmul(S2,np.transpose(P))
          #6 Find generalized eigenvector and eigenvalues for S1 and S2
          eigvalB,B=LAS.eig(S1,S2, left=False, right=True)
          #7 Find W
          eigvalB=np.sort(eigvalB) #arranging in ascending to have the lowest eigenvalue on top and highest on bottom
          B_ind=np.argsort(eigvalB)
          B=B[:,B_ind]
          W=np.matmul(np.transpose(B),P)
          return W

  W1=CSP_projectionMat(c1_feat_Train_1,c2_feat_Train_1)
  W2=CSP_projectionMat(c1_feat_Train_2,c2_feat_Train_2)

  # TRAIN DATA
  #Finding 2 best features for each trial
  Train_x_1=np.zeros((bMEGdata_x_Train_1[0,:,0].size,2)) #bMEGdata_x_Train=(channels,trials,samples)
  Train_x_2=np.zeros((bMEGdata_x_Train_1[0,:,0].size,2))

  for trial in range(0,bMEGdata_x_Train_1[0,:,0].size): #(channels,trials,samples) size of trial
      temp=bMEGdata_x_Train_1[:,trial,:] #(channels,trials,samples)-->(channels,samples)
      Z=np.matmul(np.transpose(W1),temp) #Z: CSP Features
      Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
      Train_x_1[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

  for trial in range(0,bMEGdata_x_Train_1[0,:,0].size): #(channels,trials,samples) size of trial
      temp=bMEGdata_x_Train_2[:,trial,:] #(channels,trials,samples)-->(channels,samples)
      Z=np.matmul(np.transpose(W2),temp) #Z: CSP Features
      Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
      Train_x_2[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted


  # TEST DATA
  #Finding 2 best features for each trial

  Test_x_1=np.zeros((bMEGdata_x_Test_1[0,:,0].size,2)) #bMEGdata_x_Test=(channels,trials,samples)
  Test_x_2=np.zeros((bMEGdata_x_Test_1[0,:,0].size,2))

  for trial in range(0,bMEGdata_x_Test_1[0,:,0].size): #(channels,trials,samples) size of trial
      temp=bMEGdata_x_Test_1[:,trial,:] #(channels,trials,samples)-->(channels,samples)
      Z=np.matmul(np.transpose(W1),temp) #Z: CSP Features
      Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
      Test_x_1[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

  for trial in range(0,bMEGdata_x_Test_1[0,:,0].size): #(channels,trials,samples) size of trial
      temp=bMEGdata_x_Test_2[:,trial,:] #(channels,trials,samples)-->(channels,samples)
      Z=np.matmul(np.transpose(W2),temp) #Z: CSP Features
      Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
      Test_x_2[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

  Train_x=np.hstack([Train_x_1,Train_x_2])#,Train_x_3,Train_x_4
  Test_x=np.hstack([Test_x_1,Test_x_2])#,Test_x_3,Test_x_4
  Train_y=bMEGdata_y_Train #Y_train
  Test_y=bMEGdata_y_Test

  np.save(fname_S1+'_'+str(Classes)+'_Train_x', Train_x)
  np.save(fname_S1+'_'+str(Classes)+'_Test_x', Test_x)
  np.save(fname_S1+'_'+str(Classes)+'_Train_y', Train_y)
  np.save(fname_S1+'_'+str(Classes)+'_Test_y', Test_y)
